# <u>Drought Prediction</u>: Test Set - Resample [Mean, Max, Min], Merge

In [1]:
#Import pandas, matplotlib.pyplot, and seaborn
import pandas as pd
import numpy as np

#### Load Training Dataset and Soil Dataset.  Convert Training Dataset date variable from object to datetime.

In [2]:
# Local location of the data
# local_data = 'D:\\Data_Science\\DroughtProject\\Data\\' # Location on Windows
local_data = '/home/chad/Data/Drought_Prediction/' # Location on Linux

# Load the soil variables.
soil_set = pd.read_csv(local_data + 'soil_data.csv')

# Load the test dataset.
test_set = pd.read_csv(local_data + 'test_timeseries.csv',
                        parse_dates=['date'],
                        header=0)

# Load the validation dataset.
val_set = pd.read_csv(local_data + 'validation_timeseries.csv',
                        parse_dates=['date'],
                        header=0)

#### Confirm datasets are properly loaded and contain expected datatypes.

In [3]:
soil_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3109 entries, 0 to 3108
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   fips           3109 non-null   int64  
 1   lat            3109 non-null   float64
 2   lon            3109 non-null   float64
 3   elevation      3109 non-null   int64  
 4   slope1         3109 non-null   float64
 5   slope2         3109 non-null   float64
 6   slope3         3109 non-null   float64
 7   slope4         3109 non-null   float64
 8   slope5         3109 non-null   float64
 9   slope6         3109 non-null   float64
 10  slope7         3109 non-null   float64
 11  slope8         3109 non-null   float64
 12  aspectN        3109 non-null   float64
 13  aspectE        3109 non-null   float64
 14  aspectS        3109 non-null   float64
 15  aspectW        3109 non-null   float64
 16  aspectUnknown  3109 non-null   float64
 17  WAT_LAND       3109 non-null   float64
 18  NVG_LAND

In [4]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2271948 entries, 0 to 2271947
Data columns (total 21 columns):
 #   Column       Dtype         
---  ------       -----         
 0   fips         int64         
 1   date         datetime64[ns]
 2   PRECTOT      float64       
 3   PS           float64       
 4   QV2M         float64       
 5   T2M          float64       
 6   T2MDEW       float64       
 7   T2MWET       float64       
 8   T2M_MAX      float64       
 9   T2M_MIN      float64       
 10  T2M_RANGE    float64       
 11  TS           float64       
 12  WS10M        float64       
 13  WS10M_MAX    float64       
 14  WS10M_MIN    float64       
 15  WS10M_RANGE  float64       
 16  WS50M        float64       
 17  WS50M_MAX    float64       
 18  WS50M_MIN    float64       
 19  WS50M_RANGE  float64       
 20  score        float64       
dtypes: datetime64[ns](1), float64(19), int64(1)
memory usage: 364.0 MB


In [5]:
val_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2268840 entries, 0 to 2268839
Data columns (total 21 columns):
 #   Column       Dtype         
---  ------       -----         
 0   fips         int64         
 1   date         datetime64[ns]
 2   PRECTOT      float64       
 3   PS           float64       
 4   QV2M         float64       
 5   T2M          float64       
 6   T2MDEW       float64       
 7   T2MWET       float64       
 8   T2M_MAX      float64       
 9   T2M_MIN      float64       
 10  T2M_RANGE    float64       
 11  TS           float64       
 12  WS10M        float64       
 13  WS10M_MAX    float64       
 14  WS10M_MIN    float64       
 15  WS10M_RANGE  float64       
 16  WS50M        float64       
 17  WS50M_MAX    float64       
 18  WS50M_MIN    float64       
 19  WS50M_RANGE  float64       
 20  score        float64       
dtypes: datetime64[ns](1), float64(19), int64(1)
memory usage: 363.5 MB


In [6]:
soil_set.head()

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
1,1005,31.870670,-85.405456,146,0.0158,0.1868,0.5441,0.2424,0.0106,0.0003,...,72.578804,1.828159,74.406960,3,2,1,1,1,1,1
2,1003,30.659218,-87.746067,52,0.0746,0.4370,0.4415,0.0469,0.0000,0.0000,...,59.843639,2.996914,62.840553,3,2,1,2,1,1,1
3,1007,33.015893,-87.127148,93,0.0144,0.1617,0.3714,0.3493,0.0898,0.0134,...,1.916593,0.008330,1.924924,3,2,1,1,1,1,1
4,1009,33.977448,-86.567246,198,0.0050,0.0872,0.2799,0.3576,0.1477,0.1037,...,1.891909,0.027488,1.919397,3,2,1,1,1,1,1


In [7]:
test_set.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2019-01-01,2.25,100.51,9.69,14.71,13.55,13.52,17.38,10.92,...,14.63,1.20,1.50,0.79,0.71,2.74,4.01,1.23,2.78,0.0
1,1001,2019-01-02,4.94,100.48,8.65,13.05,11.83,11.74,17.76,9.54,...,13.06,1.02,1.35,0.32,1.03,2.13,3.37,0.56,2.81,NaN
2,1001,2019-01-03,20.74,100.03,8.59,12.12,11.67,11.67,13.74,10.44,...,12.12,1.83,4.23,0.34,3.88,3.41,7.49,0.72,6.77,NaN
3,1001,2019-01-04,16.17,99.47,7.93,11.80,10.38,9.59,18.19,5.44,...,11.72,3.75,5.25,2.34,2.91,6.78,9.61,4.90,4.71,NaN
4,1001,2019-01-05,0.01,100.39,5.32,7.91,4.73,4.71,14.37,3.68,...,7.75,2.32,2.90,1.69,1.21,4.42,5.87,3.15,2.72,NaN


In [8]:
val_set.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2017-01-01,32.50,100.02,10.47,14.69,14.47,14.47,17.68,10.53,...,14.63,2.14,2.71,1.52,1.19,4.40,5.96,2.25,3.71,NaN
1,1001,2017-01-02,63.52,100.04,12.75,17.96,17.75,17.75,20.30,16.14,...,17.85,2.75,4.31,1.60,2.71,5.50,8.16,4.05,4.11,NaN
2,1001,2017-01-03,18.82,99.69,9.74,14.24,13.44,13.44,18.48,9.29,...,14.06,2.25,3.73,1.64,2.09,4.80,7.27,2.54,4.72,2.0
3,1001,2017-01-04,0.01,100.02,5.21,8.10,3.86,3.88,11.74,2.12,...,8.08,2.63,3.95,1.34,2.60,4.98,6.16,3.36,2.80,NaN
4,1001,2017-01-05,0.01,99.89,4.54,5.91,2.20,2.22,13.07,-0.18,...,5.85,1.76,2.76,0.47,2.28,3.43,4.70,0.66,4.04,NaN


## Combine Test & Validation Datasets

In [9]:
# Using concat to merge test and validation datasets.
testval = pd.concat([val_set, test_set], axis=0, ignore_index=True)

In [10]:
testval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4540788 entries, 0 to 4540787
Data columns (total 21 columns):
 #   Column       Dtype         
---  ------       -----         
 0   fips         int64         
 1   date         datetime64[ns]
 2   PRECTOT      float64       
 3   PS           float64       
 4   QV2M         float64       
 5   T2M          float64       
 6   T2MDEW       float64       
 7   T2MWET       float64       
 8   T2M_MAX      float64       
 9   T2M_MIN      float64       
 10  T2M_RANGE    float64       
 11  TS           float64       
 12  WS10M        float64       
 13  WS10M_MAX    float64       
 14  WS10M_MIN    float64       
 15  WS10M_RANGE  float64       
 16  WS50M        float64       
 17  WS50M_MAX    float64       
 18  WS50M_MIN    float64       
 19  WS50M_RANGE  float64       
 20  score        float64       
dtypes: datetime64[ns](1), float64(19), int64(1)
memory usage: 727.5 MB


In [11]:
testval

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2017-01-01,32.50,100.02,10.47,14.69,14.47,14.47,17.68,10.53,...,14.63,2.14,2.71,1.52,1.19,4.40,5.96,2.25,3.71,NaN
1,1001,2017-01-02,63.52,100.04,12.75,17.96,17.75,17.75,20.30,16.14,...,17.85,2.75,4.31,1.60,2.71,5.50,8.16,4.05,4.11,NaN
2,1001,2017-01-03,18.82,99.69,9.74,14.24,13.44,13.44,18.48,9.29,...,14.06,2.25,3.73,1.64,2.09,4.80,7.27,2.54,4.72,2.0000
3,1001,2017-01-04,0.01,100.02,5.21,8.10,3.86,3.88,11.74,2.12,...,8.08,2.63,3.95,1.34,2.60,4.98,6.16,3.36,2.80,NaN
4,1001,2017-01-05,0.01,99.89,4.54,5.91,2.20,2.22,13.07,-0.18,...,5.85,1.76,2.76,0.47,2.28,3.43,4.70,0.66,4.04,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4540783,56043,2020-12-27,3.83,82.91,2.77,-3.61,-6.77,-5.19,0.24,-9.07,...,-5.88,2.98,6.04,0.72,5.31,4.56,7.68,1.03,6.66,NaN
4540784,56043,2020-12-28,0.00,83.04,1.82,-7.31,-12.06,-9.68,-1.48,-11.51,...,-10.61,1.83,2.81,0.10,2.72,2.52,3.69,0.14,3.56,NaN
4540785,56043,2020-12-29,0.00,82.78,1.87,-7.38,-11.79,-9.59,-0.88,-11.39,...,-10.81,2.70,6.22,0.18,6.04,3.84,7.66,0.29,7.37,3.5182
4540786,56043,2020-12-30,0.01,82.87,1.57,-6.40,-13.94,-10.17,1.33,-12.16,...,-8.74,2.93,4.10,1.62,2.48,4.42,6.05,2.79,3.26,NaN


### Resample Meteorolgical Variables to weekly variables with non-null Score values.

In [12]:
# For each county ('fips'), since the score value is set on Tuesday, 
# all variables are averaged from the week leading up to Tuesday: previous Wednesday to Tuesday.
testval_mean = test_set.groupby('fips').resample('W-TUE', on='date').mean()
testval_mean

fips   PRECTOT          PS      QV2M        T2M  \
fips  date                                                             
1001  2019-01-01   1001.0  2.250000  100.510000  9.690000  14.710000   
      2019-01-08   1001.0  5.988571  100.325714  7.432857  11.331429   
      2019-01-15   1001.0  1.595714  100.844286  4.768571   5.620000   
      2019-01-22   1001.0  4.118571  100.597143  5.472857   6.492857   
      2019-01-29   1001.0  8.361429  100.404286  4.734286   5.395714   
...                   ...       ...         ...       ...        ...   
56043 2020-12-08  56043.0  0.000000   83.890000  1.895714  -1.410000   
      2020-12-15  56043.0  1.014286   82.878571  2.211429  -4.488571   
      2020-12-22  56043.0  0.800000   82.717143  2.850000  -1.897143   
      2020-12-29  56043.0  0.755714   83.038571  2.067143  -5.272857   
      2021-01-05  56043.0  0.005000   82.845000  1.850000  -5.115000   

                     T2MDEW     T2MWET    T2M_MAX    T2M_MIN  T2M_RANGE  \
fips  date                                                                
1001  2019-01-01  13.550000  13.520000  17.380000  10.920000   6.460000   
      2019-01-08   9.334286   9.168571  17.271429   6.542857  10.730000   
      2019-01-15   2.870000   2.715714  11.077143   1.025714  10.050000   
      2019-01-22   3.995714   3.752857  12.291429   0.280000  12.012857   
      2019-01-29   2.294286   2.107143  11.401429  -0.671429  12.072857   
...                     ...        ...        ...        ...        ...   
56043 2020-12-08 -11.617143  -6.515714   5.211429  -5.534286  10.744286   
      2020-12-15  -9.947143  -7.217143  -0.511429  -7.604286   7.091429   
      2020-12-22  -6.667143  -4.281429   4.004286  -6.581429  10.587143   
      2020-12-29 -10.781429  -8.025714   0.561429  -9.497143  10.058571   
      2021-01-05 -12.030000  -8.575000   1.745000 -10.365000  12.110000   

                         TS     WS10M  WS10M_MAX  WS10M_MIN  WS10M_RANGE  \
fips  date                                                                 
1001  2019-01-01  14.630000  1.200000   1.500000   0.790000     0.710000   
      2019-01-08  11.012857  2.080000   3.100000   1.124286     1.974286   
      2019-01-15   5.602857  2.372857   3.214286   1.718571     1.495714   
      2019-01-22   6.104286  2.644286   3.752857   1.468571     2.285714   
      2019-01-29   4.821429  2.561429   3.828571   1.430000     2.397143   
...                     ...       ...        ...        ...          ...   
56043 2020-12-08  -4.058571  2.024286   2.814286   1.150000     1.665714   
      2020-12-15  -5.275714  2.917143   4.657143   1.302857     3.352857   
      2020-12-22  -3.534286  5.058571   7.348571   3.462857     3.885714   
      2020-12-29  -7.225714  3.734286   5.957143   1.904286     4.051429   
      2021-01-05  -7.675000  2.545000   3.910000   0.965000     2.945000   

                     WS50M  WS50M_MAX  WS50M_MIN  WS50M_RANGE   score  
fips  date                                                             
1001  2019-01-01  2.740000   4.010000   1.230000     2.780000  0.0000  
      2019-01-08  4.075714   6.244286   2.172857     4.071429  0.0000  
      2019-01-15  4.565714   5.957143   3.401429     2.557143  0.0000  
      2019-01-22  4.828571   7.045714   2.884286     4.161429  0.0000  
      2019-01-29  4.658571   6.860000   2.715714     4.144286  0.0000  
...                    ...        ...        ...          ...     ...  
56043 2020-12-08  3.084286   4.588571   1.758571     2.830000  3.5109  
      2020-12-15  4.342857   6.597143   2.017143     4.581429  3.5182  
      2020-12-22  7.721429  10.355714   5.382857     4.970000  3.5182  
      2020-12-29  5.610000   8.338571   2.992857     5.347143  3.5182  
      2021-01-05  3.905000   6.255000   1.600000     4.660000     NaN  

[329448 rows x 20 columns]

In [13]:
# The Groupby returns a MultiIndex.
testval_mean.index

MultiIndex([( 1001, '2019-01-01'),
            ( 1001, '2019-01-08'),
            ( 1001, '2019-01-15'),
            ( 1001, '2019-01-22'),
            ( 1001, '2019-01-29'),
            ( 1001, '2019-02-05'),
            ( 1001, '2019-02-12'),
            ( 1001, '2019-02-19'),
            ( 1001, '2019-02-26'),
            ( 1001, '2019-03-05'),
            ...
            (56043, '2020-11-03'),
            (56043, '2020-11-10'),
            (56043, '2020-11-17'),
            (56043, '2020-11-24'),
            (56043, '2020-12-01'),
            (56043, '2020-12-08'),
            (56043, '2020-12-15'),
            (56043, '2020-12-22'),
            (56043, '2020-12-29'),
            (56043, '2021-01-05')],
           names=['fips', 'date'], length=329448)

In [14]:
# 'fips' is both part of the MultiIndex and a copied column.
#  Need to rename (or delete) before resetting the index.
testval_mean.rename({'fips': 'fips_copy'}, axis=1, inplace=True)

testval_mean.reset_index(inplace=True)
testval_mean

,fips,date,fips_copy,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2019-01-01,1001.0,2.250000,100.510000,9.690000,14.710000,13.550000,13.520000,17.380000,...,14.630000,1.200000,1.500000,0.790000,0.710000,2.740000,4.010000,1.230000,2.780000,0.0000
1,1001,2019-01-08,1001.0,5.988571,100.325714,7.432857,11.331429,9.334286,9.168571,17.271429,...,11.012857,2.080000,3.100000,1.124286,1.974286,4.075714,6.244286,2.172857,4.071429,0.0000
2,1001,2019-01-15,1001.0,1.595714,100.844286,4.768571,5.620000,2.870000,2.715714,11.077143,...,5.602857,2.372857,3.214286,1.718571,1.495714,4.565714,5.957143,3.401429,2.557143,0.0000
3,1001,2019-01-22,1001.0,4.118571,100.597143,5.472857,6.492857,3.995714,3.752857,12.291429,...,6.104286,2.644286,3.752857,1.468571,2.285714,4.828571,7.045714,2.884286,4.161429,0.0000
4,1001,2019-01-29,1001.0,8.361429,100.404286,4.734286,5.395714,2.294286,2.107143,11.401429,...,4.821429,2.561429,3.828571,1.430000,2.397143,4.658571,6.860000,2.715714,4.144286,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329443,56043,2020-12-08,56043.0,0.000000,83.890000,1.895714,-1.410000,-11.617143,-6.515714,5.211429,...,-4.058571,2.024286,2.814286,1.150000,1.665714,3.084286,4.588571,1.758571,2.830000,3.5109
329444,56043,2020-12-15,56043.0,1.014286,82.878571,2.211429,-4.488571,-9.947143,-7.217143,-0.511429,...,-5.275714,2.917143,4.657143,1.302857,3.352857,4.342857,6.597143,2.017143,4.581429,3.5182
329445,56043,2020-12-22,56043.0,0.800000,82.717143,2.850000,-1.897143,-6.667143,-4.281429,4.004286,...,-3.534286,5.058571,7.348571,3.462857,3.885714,7.721429,10.355714,5.382857,4.970000,3.5182
329446,56043,2020-12-29,56043.0,0.755714,83.038571,2.067143,-5.272857,-10.781429,-8.025714,0.561429,...,-7.225714,3.734286,5.957143,1.904286,4.051429,5.610000,8.338571,2.992857,5.347143,3.5182


In [15]:
# Confirming expected column dataypes, overall size, memory usage, etc.
testval_mean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329448 entries, 0 to 329447
Data columns (total 22 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   fips         329448 non-null  int64         
 1   date         329448 non-null  datetime64[ns]
 2   fips_copy    329448 non-null  float64       
 3   PRECTOT      329448 non-null  float64       
 4   PS           329448 non-null  float64       
 5   QV2M         329448 non-null  float64       
 6   T2M          329448 non-null  float64       
 7   T2MDEW       329448 non-null  float64       
 8   T2MWET       329448 non-null  float64       
 9   T2M_MAX      329448 non-null  float64       
 10  T2M_MIN      329448 non-null  float64       
 11  T2M_RANGE    329448 non-null  float64       
 12  TS           329448 non-null  float64       
 13  WS10M        329448 non-null  float64       
 14  WS10M_MAX    329448 non-null  float64       
 15  WS10M_MIN    329448 non-null  floa

In [16]:
# The last Score value is NaN.  Filling that value with last valid value.
testval_mean.fillna(method='ffill', inplace=True)

In [17]:
# 'fips_copy' is a copy and has been verified as no longer needed.
testval_mean.drop('fips_copy', axis=1, inplace=True)

In [18]:
# Confirming proper structure and expected output.
testval_mean

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2019-01-01,2.250000,100.510000,9.690000,14.710000,13.550000,13.520000,17.380000,10.920000,...,14.630000,1.200000,1.500000,0.790000,0.710000,2.740000,4.010000,1.230000,2.780000,0.0000
1,1001,2019-01-08,5.988571,100.325714,7.432857,11.331429,9.334286,9.168571,17.271429,6.542857,...,11.012857,2.080000,3.100000,1.124286,1.974286,4.075714,6.244286,2.172857,4.071429,0.0000
2,1001,2019-01-15,1.595714,100.844286,4.768571,5.620000,2.870000,2.715714,11.077143,1.025714,...,5.602857,2.372857,3.214286,1.718571,1.495714,4.565714,5.957143,3.401429,2.557143,0.0000
3,1001,2019-01-22,4.118571,100.597143,5.472857,6.492857,3.995714,3.752857,12.291429,0.280000,...,6.104286,2.644286,3.752857,1.468571,2.285714,4.828571,7.045714,2.884286,4.161429,0.0000
4,1001,2019-01-29,8.361429,100.404286,4.734286,5.395714,2.294286,2.107143,11.401429,-0.671429,...,4.821429,2.561429,3.828571,1.430000,2.397143,4.658571,6.860000,2.715714,4.144286,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329443,56043,2020-12-08,0.000000,83.890000,1.895714,-1.410000,-11.617143,-6.515714,5.211429,-5.534286,...,-4.058571,2.024286,2.814286,1.150000,1.665714,3.084286,4.588571,1.758571,2.830000,3.5109
329444,56043,2020-12-15,1.014286,82.878571,2.211429,-4.488571,-9.947143,-7.217143,-0.511429,-7.604286,...,-5.275714,2.917143,4.657143,1.302857,3.352857,4.342857,6.597143,2.017143,4.581429,3.5182
329445,56043,2020-12-22,0.800000,82.717143,2.850000,-1.897143,-6.667143,-4.281429,4.004286,-6.581429,...,-3.534286,5.058571,7.348571,3.462857,3.885714,7.721429,10.355714,5.382857,4.970000,3.5182
329446,56043,2020-12-29,0.755714,83.038571,2.067143,-5.272857,-10.781429,-8.025714,0.561429,-9.497143,...,-7.225714,3.734286,5.957143,1.904286,4.051429,5.610000,8.338571,2.992857,5.347143,3.5182


### Repeat process of resampling but use the max() value instead of mean()

In [20]:
# For each county ('fips'), since the score value is set on Tuesday, 
# find the max for all variables from the week leading up to Tuesday: previous Wednesday to Tuesday.
testval_max = testval.groupby('fips').resample('W-TUE', on='date').max()

testval_max.rename({'fips': 'fips_copy', 'date': 'date_copy'}, axis=1, inplace=True)
testval_max.reset_index(inplace=True)
testval_max.fillna(method='ffill', inplace=True)
testval_max.drop(['fips_copy', 'date_copy'], axis=1, inplace=True)

testval_max

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2017-01-03,63.52,100.04,12.75,17.96,17.75,17.75,20.30,16.14,...,17.85,2.75,4.31,1.64,2.71,5.50,8.16,4.05,4.72,2.0000
1,1001,2017-01-10,19.85,102.36,5.62,8.10,4.54,4.55,13.07,2.12,...,8.08,3.64,4.47,2.84,2.60,6.51,7.68,4.75,4.23,2.0000
2,1001,2017-01-17,0.33,101.50,11.41,17.21,16.06,16.06,23.88,13.15,...,17.10,2.88,4.24,2.26,1.98,5.73,7.01,4.76,5.41,2.0000
3,1001,2017-01-24,37.95,100.49,12.76,18.19,17.63,17.63,22.51,17.13,...,18.10,4.44,8.13,2.35,6.52,7.79,12.19,5.03,9.49,1.0000
4,1001,2017-01-31,8.16,100.60,9.00,13.37,11.81,11.81,20.29,5.05,...,13.27,2.98,4.70,2.02,3.14,5.70,7.71,4.78,5.28,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652675,56043,2020-12-08,0.00,84.19,2.13,1.32,-10.05,-4.67,8.08,-3.14,...,-1.57,3.05,3.85,2.36,3.04,4.73,6.30,3.85,4.93,3.5109
652676,56043,2020-12-15,3.19,83.09,3.18,1.71,-4.94,-3.39,8.64,-1.54,...,0.00,3.97,6.88,1.92,5.59,6.39,9.41,2.83,8.08,3.5182
652677,56043,2020-12-22,2.97,83.05,4.04,1.39,-1.77,-0.19,6.87,-1.73,...,-0.23,8.83,13.78,6.64,7.14,11.90,17.31,9.05,8.25,3.5182
652678,56043,2020-12-29,3.83,83.57,2.77,-1.78,-6.77,-4.57,4.54,-5.28,...,-3.20,6.67,12.19,3.83,8.36,9.23,15.39,6.40,8.98,3.5182


### Repeat process of resampling but use the min() value instead of mean()

In [21]:
# CREATE FUNCTION FOR THIS PROCESS

# For each county ('fips'), since the score value is set on Tuesday, 
# find the max for all variables from the week leading up to Tuesday: previous Wednesday to Tuesday.
testval_min = testval.groupby('fips').resample('W-TUE', on='date').min()

testval_min.rename({'fips': 'fips_copy', 'date': 'date_copy'}, axis=1, inplace=True)
testval_min.reset_index(inplace=True)
testval_min.fillna(method='ffill', inplace=True)
testval_min.drop(['fips_copy', 'date_copy'], axis=1, inplace=True)

testval_min

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2017-01-03,18.82,99.69,9.74,14.24,13.44,13.44,17.68,9.29,...,14.06,2.14,2.71,1.52,1.19,4.40,5.96,2.25,3.71,2.0000
1,1001,2017-01-10,0.00,99.89,1.80,-6.49,-9.40,-9.28,-1.02,-12.89,...,-8.36,1.76,2.74,0.47,1.04,3.43,4.66,0.66,2.21,2.0000
2,1001,2017-01-17,0.00,100.60,8.73,14.38,12.09,12.09,20.24,9.97,...,14.21,0.95,2.09,0.36,1.24,1.43,3.76,0.53,2.25,2.0000
3,1001,2017-01-24,0.00,97.97,5.60,9.27,5.34,5.35,16.23,4.18,...,8.95,1.48,1.99,0.88,1.11,3.03,4.86,1.33,3.53,1.0000
4,1001,2017-01-31,0.00,99.57,3.58,3.63,-0.85,-0.81,8.21,-2.01,...,3.73,2.11,2.83,1.01,1.24,4.65,6.37,1.71,1.59,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652675,56043,2020-12-08,0.00,83.70,1.45,-6.39,-14.85,-9.39,-1.29,-9.30,...,-7.18,1.20,1.76,0.08,1.04,1.73,2.82,0.11,1.83,3.5109
652676,56043,2020-12-15,0.00,82.62,1.41,-7.92,-15.19,-11.55,-4.30,-12.97,...,-8.79,2.08,2.79,0.57,1.57,3.01,3.84,1.06,2.01,3.5182
652677,56043,2020-12-22,0.00,82.00,2.11,-3.90,-10.25,-7.07,1.08,-8.40,...,-6.40,2.89,4.11,1.35,2.76,4.66,7.80,1.99,3.97,3.5182
652678,56043,2020-12-29,0.00,82.50,1.36,-8.03,-15.58,-11.80,-4.04,-11.51,...,-10.81,1.83,2.81,0.10,1.50,2.52,3.69,0.14,3.32,3.5182


### Merge Mean, Min, & Max Resample

In [23]:
# Add '_mean' suffix so when the tables are joined, the variable suffixes have a standard meaning.
testval_mean = testval_mean.add_suffix('_mean')
testval_max = testval_max.add_suffix('_max')
testval_min = testval_min.add_suffix('_min')

In [24]:
testval_stats_temp = testval_mean.join(testval_max, how='inner', rsuffix = '_max')
testval_stats_temp

,fips_mean,date_mean,PRECTOT_mean,PS_mean,QV2M_mean,T2M_mean,T2MDEW_mean,T2MWET_mean,T2M_MAX_mean,T2M_MIN_mean,...,TS_max,WS10M_max,WS10M_MAX_max,WS10M_MIN_max,WS10M_RANGE_max,WS50M_max,WS50M_MAX_max,WS50M_MIN_max,WS50M_RANGE_max,score_max
0,1001,2019-01-01,2.250000,100.510000,9.690000,14.710000,13.550000,13.520000,17.380000,10.920000,...,17.85,2.75,4.31,1.64,2.71,5.50,8.16,4.05,4.72,2.0
1,1001,2019-01-08,5.988571,100.325714,7.432857,11.331429,9.334286,9.168571,17.271429,6.542857,...,8.08,3.64,4.47,2.84,2.60,6.51,7.68,4.75,4.23,2.0
2,1001,2019-01-15,1.595714,100.844286,4.768571,5.620000,2.870000,2.715714,11.077143,1.025714,...,17.10,2.88,4.24,2.26,1.98,5.73,7.01,4.76,5.41,2.0
3,1001,2019-01-22,4.118571,100.597143,5.472857,6.492857,3.995714,3.752857,12.291429,0.280000,...,18.10,4.44,8.13,2.35,6.52,7.79,12.19,5.03,9.49,1.0
4,1001,2019-01-29,8.361429,100.404286,4.734286,5.395714,2.294286,2.107143,11.401429,-0.671429,...,13.27,2.98,4.70,2.02,3.14,5.70,7.71,4.78,5.28,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329443,56043,2020-12-08,0.000000,83.890000,1.895714,-1.410000,-11.617143,-6.515714,5.211429,-5.534286,...,-4.00,6.89,9.62,4.26,7.77,10.55,12.88,8.04,10.52,0.0
329444,56043,2020-12-15,1.014286,82.878571,2.211429,-4.488571,-9.947143,-7.217143,-0.511429,-7.604286,...,-1.90,8.17,10.98,4.21,8.03,10.79,13.77,6.91,11.40,0.0
329445,56043,2020-12-22,0.800000,82.717143,2.850000,-1.897143,-6.667143,-4.281429,4.004286,-6.581429,...,1.45,10.03,14.18,5.96,8.23,13.75,18.27,9.88,8.40,0.0
329446,56043,2020-12-29,0.755714,83.038571,2.067143,-5.272857,-10.781429,-8.025714,0.561429,-9.497143,...,4.80,9.46,12.86,4.53,8.33,13.13,16.70,8.16,11.21,0.0


In [25]:
testval_stats =  testval_stats_temp.join(testval_min, how='inner', rsuffix='_min')
testval_stats

,fips_mean,date_mean,PRECTOT_mean,PS_mean,QV2M_mean,T2M_mean,T2MDEW_mean,T2MWET_mean,T2M_MAX_mean,T2M_MIN_mean,...,TS_min,WS10M_min,WS10M_MAX_min,WS10M_MIN_min,WS10M_RANGE_min,WS50M_min,WS50M_MAX_min,WS50M_MIN_min,WS50M_RANGE_min,score_min
0,1001,2019-01-01,2.250000,100.510000,9.690000,14.710000,13.550000,13.520000,17.380000,10.920000,...,14.06,2.14,2.71,1.52,1.19,4.40,5.96,2.25,3.71,2.0
1,1001,2019-01-08,5.988571,100.325714,7.432857,11.331429,9.334286,9.168571,17.271429,6.542857,...,-8.36,1.76,2.74,0.47,1.04,3.43,4.66,0.66,2.21,2.0
2,1001,2019-01-15,1.595714,100.844286,4.768571,5.620000,2.870000,2.715714,11.077143,1.025714,...,14.21,0.95,2.09,0.36,1.24,1.43,3.76,0.53,2.25,2.0
3,1001,2019-01-22,4.118571,100.597143,5.472857,6.492857,3.995714,3.752857,12.291429,0.280000,...,8.95,1.48,1.99,0.88,1.11,3.03,4.86,1.33,3.53,1.0
4,1001,2019-01-29,8.361429,100.404286,4.734286,5.395714,2.294286,2.107143,11.401429,-0.671429,...,3.73,2.11,2.83,1.01,1.24,4.65,6.37,1.71,1.59,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329443,56043,2020-12-08,0.000000,83.890000,1.895714,-1.410000,-11.617143,-6.515714,5.211429,-5.534286,...,-11.18,2.85,5.01,0.41,2.62,3.90,6.27,0.57,3.04,0.0
329444,56043,2020-12-15,1.014286,82.878571,2.211429,-4.488571,-9.947143,-7.217143,-0.511429,-7.604286,...,-12.04,2.43,5.26,0.14,3.87,3.46,7.95,0.17,3.49,0.0
329445,56043,2020-12-22,0.800000,82.717143,2.850000,-1.897143,-6.667143,-4.281429,4.004286,-6.581429,...,-3.57,3.49,5.43,0.37,2.81,4.96,8.58,0.50,2.99,0.0
329446,56043,2020-12-29,0.755714,83.038571,2.067143,-5.272857,-10.781429,-8.025714,0.561429,-9.497143,...,-0.70,2.15,3.06,0.58,2.15,2.83,5.38,0.94,3.80,0.0


In [26]:
# The Date, FIPS, and Score values don't have min or max values different from mean and are therefore duplicates.
testval_stats.drop(['fips_max', 'date_max', 'score_max', 'fips_min', 'date_min', 'score_min'], axis=1, inplace=True)

In [27]:
testval_stats

,fips_mean,date_mean,PRECTOT_mean,PS_mean,QV2M_mean,T2M_mean,T2MDEW_mean,T2MWET_mean,T2M_MAX_mean,T2M_MIN_mean,...,T2M_RANGE_min,TS_min,WS10M_min,WS10M_MAX_min,WS10M_MIN_min,WS10M_RANGE_min,WS50M_min,WS50M_MAX_min,WS50M_MIN_min,WS50M_RANGE_min
0,1001,2019-01-01,2.250000,100.510000,9.690000,14.710000,13.550000,13.520000,17.380000,10.920000,...,4.16,14.06,2.14,2.71,1.52,1.19,4.40,5.96,2.25,3.71
1,1001,2019-01-08,5.988571,100.325714,7.432857,11.331429,9.334286,9.168571,17.271429,6.542857,...,8.99,-8.36,1.76,2.74,0.47,1.04,3.43,4.66,0.66,2.21
2,1001,2019-01-15,1.595714,100.844286,4.768571,5.620000,2.870000,2.715714,11.077143,1.025714,...,9.56,14.21,0.95,2.09,0.36,1.24,1.43,3.76,0.53,2.25
3,1001,2019-01-22,4.118571,100.597143,5.472857,6.492857,3.995714,3.752857,12.291429,0.280000,...,3.75,8.95,1.48,1.99,0.88,1.11,3.03,4.86,1.33,3.53
4,1001,2019-01-29,8.361429,100.404286,4.734286,5.395714,2.294286,2.107143,11.401429,-0.671429,...,8.17,3.73,2.11,2.83,1.01,1.24,4.65,6.37,1.71,1.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329443,56043,2020-12-08,0.000000,83.890000,1.895714,-1.410000,-11.617143,-6.515714,5.211429,-5.534286,...,6.41,-11.18,2.85,5.01,0.41,2.62,3.90,6.27,0.57,3.04
329444,56043,2020-12-15,1.014286,82.878571,2.211429,-4.488571,-9.947143,-7.217143,-0.511429,-7.604286,...,8.10,-12.04,2.43,5.26,0.14,3.87,3.46,7.95,0.17,3.49
329445,56043,2020-12-22,0.800000,82.717143,2.850000,-1.897143,-6.667143,-4.281429,4.004286,-6.581429,...,5.73,-3.57,3.49,5.43,0.37,2.81,4.96,8.58,0.50,2.99
329446,56043,2020-12-29,0.755714,83.038571,2.067143,-5.272857,-10.781429,-8.025714,0.561429,-9.497143,...,8.54,-0.70,2.15,3.06,0.58,2.15,2.83,5.38,0.94,3.80


In [28]:
testval_stats.rename({'fips_mean': 'fips', 'date_mean': 'date', 'score_mean':'score'}, axis=1, inplace=True)

In [29]:
testval_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329448 entries, 0 to 329447
Data columns (total 57 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   fips              329448 non-null  int64         
 1   date              329448 non-null  datetime64[ns]
 2   PRECTOT_mean      329448 non-null  float64       
 3   PS_mean           329448 non-null  float64       
 4   QV2M_mean         329448 non-null  float64       
 5   T2M_mean          329448 non-null  float64       
 6   T2MDEW_mean       329448 non-null  float64       
 7   T2MWET_mean       329448 non-null  float64       
 8   T2M_MAX_mean      329448 non-null  float64       
 9   T2M_MIN_mean      329448 non-null  float64       
 10  T2M_RANGE_mean    329448 non-null  float64       
 11  TS_mean           329448 non-null  float64       
 12  WS10M_mean        329448 non-null  float64       
 13  WS10M_MAX_mean    329448 non-null  float64       
 14  WS10

In [30]:
# Rearranging the columns so that fips, date, and score are the first three columns.

cols = testval_stats.columns.tolist()
cols = cols[0:2] + [cols[20]] + cols[2:20] + cols[21:]
# type(cols)
cols

['fips',
 'date',
 'score',
 'PRECTOT_mean',
 'PS_mean',
 'QV2M_mean',
 'T2M_mean',
 'T2MDEW_mean',
 'T2MWET_mean',
 'T2M_MAX_mean',
 'T2M_MIN_mean',
 'T2M_RANGE_mean',
 'TS_mean',
 'WS10M_mean',
 'WS10M_MAX_mean',
 'WS10M_MIN_mean',
 'WS10M_RANGE_mean',
 'WS50M_mean',
 'WS50M_MAX_mean',
 'WS50M_MIN_mean',
 'WS50M_RANGE_mean',
 'PRECTOT_max',
 'PS_max',
 'QV2M_max',
 'T2M_max',
 'T2MDEW_max',
 'T2MWET_max',
 'T2M_MAX_max',
 'T2M_MIN_max',
 'T2M_RANGE_max',
 'TS_max',
 'WS10M_max',
 'WS10M_MAX_max',
 'WS10M_MIN_max',
 'WS10M_RANGE_max',
 'WS50M_max',
 'WS50M_MAX_max',
 'WS50M_MIN_max',
 'WS50M_RANGE_max',
 'PRECTOT_min',
 'PS_min',
 'QV2M_min',
 'T2M_min',
 'T2MDEW_min',
 'T2MWET_min',
 'T2M_MAX_min',
 'T2M_MIN_min',
 'T2M_RANGE_min',
 'TS_min',
 'WS10M_min',
 'WS10M_MAX_min',
 'WS10M_MIN_min',
 'WS10M_RANGE_min',
 'WS50M_min',
 'WS50M_MAX_min',
 'WS50M_MIN_min',
 'WS50M_RANGE_min']

In [31]:
testval_stats = testval_stats[cols]
testval_stats

,fips,date,score,PRECTOT_mean,PS_mean,QV2M_mean,T2M_mean,T2MDEW_mean,T2MWET_mean,T2M_MAX_mean,...,T2M_RANGE_min,TS_min,WS10M_min,WS10M_MAX_min,WS10M_MIN_min,WS10M_RANGE_min,WS50M_min,WS50M_MAX_min,WS50M_MIN_min,WS50M_RANGE_min
0,1001,2019-01-01,0.0000,2.250000,100.510000,9.690000,14.710000,13.550000,13.520000,17.380000,...,4.16,14.06,2.14,2.71,1.52,1.19,4.40,5.96,2.25,3.71
1,1001,2019-01-08,0.0000,5.988571,100.325714,7.432857,11.331429,9.334286,9.168571,17.271429,...,8.99,-8.36,1.76,2.74,0.47,1.04,3.43,4.66,0.66,2.21
2,1001,2019-01-15,0.0000,1.595714,100.844286,4.768571,5.620000,2.870000,2.715714,11.077143,...,9.56,14.21,0.95,2.09,0.36,1.24,1.43,3.76,0.53,2.25
3,1001,2019-01-22,0.0000,4.118571,100.597143,5.472857,6.492857,3.995714,3.752857,12.291429,...,3.75,8.95,1.48,1.99,0.88,1.11,3.03,4.86,1.33,3.53
4,1001,2019-01-29,0.0000,8.361429,100.404286,4.734286,5.395714,2.294286,2.107143,11.401429,...,8.17,3.73,2.11,2.83,1.01,1.24,4.65,6.37,1.71,1.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329443,56043,2020-12-08,3.5109,0.000000,83.890000,1.895714,-1.410000,-11.617143,-6.515714,5.211429,...,6.41,-11.18,2.85,5.01,0.41,2.62,3.90,6.27,0.57,3.04
329444,56043,2020-12-15,3.5182,1.014286,82.878571,2.211429,-4.488571,-9.947143,-7.217143,-0.511429,...,8.10,-12.04,2.43,5.26,0.14,3.87,3.46,7.95,0.17,3.49
329445,56043,2020-12-22,3.5182,0.800000,82.717143,2.850000,-1.897143,-6.667143,-4.281429,4.004286,...,5.73,-3.57,3.49,5.43,0.37,2.81,4.96,8.58,0.50,2.99
329446,56043,2020-12-29,3.5182,0.755714,83.038571,2.067143,-5.272857,-10.781429,-8.025714,0.561429,...,8.54,-0.70,2.15,3.06,0.58,2.15,2.83,5.38,0.94,3.80


In [32]:
testval_stats.describe()

,fips,score,PRECTOT_mean,PS_mean,QV2M_mean,T2M_mean,T2MDEW_mean,T2MWET_mean,T2M_MAX_mean,T2M_MIN_mean,...,T2M_RANGE_min,TS_min,WS10M_min,WS10M_MAX_min,WS10M_MIN_min,WS10M_RANGE_min,WS50M_min,WS50M_MAX_min,WS50M_MIN_min,WS50M_RANGE_min
count,329448.000000,329448.000000,329448.000000,329448.000000,329448.000000,329448.000000,329448.000000,329448.000000,329448.000000,329448.000000,...,329448.000000,329448.000000,329448.000000,329448.000000,329448.000000,329448.000000,329448.000000,329448.000000,329448.000000,329448.000000
mean,30670.376770,0.419171,2.949300,96.666267,8.169310,12.773807,7.649003,7.670057,18.422235,7.581988,...,7.047006,9.784691,2.122453,3.241330,0.648489,1.788345,3.391371,5.482552,0.925425,2.676349
std,14979.130122,0.846348,3.318111,5.426204,4.619144,10.522960,9.736135,9.524653,10.950231,10.248289,...,2.822341,11.747212,0.874266,1.327502,0.534245,0.857827,1.166767,1.627517,0.784674,0.976065
min,1001.000000,0.000000,0.000000,67.618571,0.300000,-28.340000,-30.750000,-28.490000,-20.970000,-34.170000,...,0.020000,-41.230000,0.260000,0.600000,0.000000,0.220000,0.460000,0.870000,0.000000,0.440000
25%,19044.500000,0.000000,0.567143,95.920000,4.205714,4.661429,0.145714,0.230000,10.292857,-0.340000,...,5.090000,0.620000,1.470000,2.220000,0.260000,1.130000,2.540000,4.300000,0.360000,1.980000
50%,29212.000000,0.000000,1.965714,98.328571,7.091429,13.424286,7.814286,7.685714,19.520000,7.681429,...,6.770000,10.340000,1.950000,2.980000,0.490000,1.620000,3.240000,5.280000,0.680000,2.500000
75%,46007.500000,0.429200,4.247143,99.945714,11.877143,21.888929,16.150000,16.020000,27.474286,16.400000,...,8.740000,19.990000,2.640000,4.050000,0.890000,2.300000,4.100000,6.510000,1.270000,3.200000
max,56043.000000,5.000000,75.595000,102.845714,21.094286,38.167143,26.170000,26.042857,46.140000,30.491429,...,21.360000,39.570000,9.040000,12.540000,7.820000,8.400000,10.800000,15.770000,9.830000,11.410000


### Directly using Merge to correctly join on specified column

In [34]:
testval_soil_stats = pd.merge(testval_stats, soil_set, on='fips', how='inner')

In [35]:
testval_soil_stats

,fips,date,score,PRECTOT_mean,PS_mean,QV2M_mean,T2M_mean,T2MDEW_mean,T2MWET_mean,T2M_MAX_mean,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,1001,2019-01-01,0.0000,2.250000,100.510000,9.690000,14.710000,13.550000,13.520000,17.380000,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
1,1001,2019-01-08,0.0000,5.988571,100.325714,7.432857,11.331429,9.334286,9.168571,17.271429,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
2,1001,2019-01-15,0.0000,1.595714,100.844286,4.768571,5.620000,2.870000,2.715714,11.077143,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
3,1001,2019-01-22,0.0000,4.118571,100.597143,5.472857,6.492857,3.995714,3.752857,12.291429,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
4,1001,2019-01-29,0.0000,8.361429,100.404286,4.734286,5.395714,2.294286,2.107143,11.401429,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329443,56043,2020-12-08,3.5109,0.000000,83.890000,1.895714,-1.410000,-11.617143,-6.515714,5.211429,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
329444,56043,2020-12-15,3.5182,1.014286,82.878571,2.211429,-4.488571,-9.947143,-7.217143,-0.511429,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
329445,56043,2020-12-22,3.5182,0.800000,82.717143,2.850000,-1.897143,-6.667143,-4.281429,4.004286,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
329446,56043,2020-12-29,3.5182,0.755714,83.038571,2.067143,-5.272857,-10.781429,-8.025714,0.561429,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1


#### There are the same number of rows in the training set and the merged dataset.

In [36]:
testval_soil_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329448 entries, 0 to 329447
Data columns (total 88 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   fips              329448 non-null  int64         
 1   date              329448 non-null  datetime64[ns]
 2   score             329448 non-null  float64       
 3   PRECTOT_mean      329448 non-null  float64       
 4   PS_mean           329448 non-null  float64       
 5   QV2M_mean         329448 non-null  float64       
 6   T2M_mean          329448 non-null  float64       
 7   T2MDEW_mean       329448 non-null  float64       
 8   T2MWET_mean       329448 non-null  float64       
 9   T2M_MAX_mean      329448 non-null  float64       
 10  T2M_MIN_mean      329448 non-null  float64       
 11  T2M_RANGE_mean    329448 non-null  float64       
 12  TS_mean           329448 non-null  float64       
 13  WS10M_mean        329448 non-null  float64       
 14  WS10

### Exporting the Merged Training and Soil Dataset 

In [37]:
# Export the merged training (meteorological) data that has been resampled with mean values
# and the soil data that does not vary with time.
testval_soil_stats.to_csv(local_data + 'testval_soil_stats.csv',
                       index_label='index')